<a href="https://colab.research.google.com/github/VickkiMars/AI-ML/blob/main/kagglebinary_competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"victorthamartian","key":"0205794fc61a8c94200ba39680afd7ca"}'}

In [3]:
!ls -lhs kaggle.json

4.0K -rw-r--r-- 1 root root 72 Aug 21 12:29 kaggle.json


In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!pwd
!chmod 600 ~/.kaggle/kaggle.json

/content


In [5]:
!kaggle competitions download -c playground-series-s4e8

100% 82.0M/82.3M [00:00<00:00, 106MB/s] 
100% 82.3M/82.3M [00:00<00:00, 87.6MB/s]


In [6]:
!ls

kaggle.json  playground-series-s4e8.zip  sample_data


In [7]:
!unzip playground-series-s4e8.zip

Archive:  playground-series-s4e8.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [8]:
pip install sklearn xgboost catboost lightgbm

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
test_df = pd.read_csv(r"/content/test.csv")
train_df = pd.read_csv(r"/content/train.csv")
sample_submission = pd.read_csv(r"/content/sample_submission.csv")

In [11]:
test = test_df
train = train_df

In [12]:
train.head()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a


In [13]:
test.head()

,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,3116945,8.64,x,NaN,n,t,NaN,NaN,w,11.13,...,b,NaN,w,u,w,t,g,NaN,d,a
1,3116946,6.90,o,t,o,f,NaN,c,y,1.27,...,NaN,NaN,n,NaN,NaN,f,f,NaN,d,a
2,3116947,2.00,b,g,n,f,NaN,c,n,6.18,...,NaN,NaN,n,NaN,NaN,f,f,NaN,d,s
3,3116948,3.47,x,t,n,f,s,c,n,4.98,...,NaN,NaN,w,NaN,n,t,z,NaN,d,u
4,3116949,6.17,x,h,y,f,p,NaN,y,6.73,...,NaN,NaN,y,NaN,y,t,NaN,NaN,d,u


In [14]:
(train.info(), test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3116945 entries, 0 to 3116944
Data columns (total 22 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   class                 object 
 2   cap-diameter          float64
 3   cap-shape             object 
 4   cap-surface           object 
 5   cap-color             object 
 6   does-bruise-or-bleed  object 
 7   gill-attachment       object 
 8   gill-spacing          object 
 9   gill-color            object 
 10  stem-height           float64
 11  stem-width            float64
 12  stem-root             object 
 13  stem-surface          object 
 14  stem-color            object 
 15  veil-type             object 
 16  veil-color            object 
 17  has-ring              object 
 18  ring-type             object 
 19  spore-print-color     object 
 20  habitat               object 
 21  season                object 
dtypes: float64(3), int64(1), object(18)
memory

(None, None)

### Correct Missing / Wrong Values

In [15]:
# Get null values data
null_data = pd.DataFrame({
    'column': train.columns,
    'null-count': train.isna().sum().values,
    '% null-count': np.round(train.isna().sum().values*100/len(train),6)
}).sort_values(by='null-count', ascending=False).reset_index(drop=True)
null_data

,column,null-count,% null-count
0,veil-type,2957493,94.884350
1,spore-print-color,2849682,91.425482
2,stem-root,2757023,88.452732
3,veil-color,2740947,87.936970
4,stem-surface,1980861,63.551362
5,gill-spacing,1258435,40.373988
6,cap-surface,671023,21.528227
7,gill-attachment,523936,16.809280
8,ring-type,128880,4.134818
9,gill-color,57,0.001829


## Drop columns with missing or duplicate values

In [19]:
cols = null_data.loc[null_data['% null-count'] > 0] # Columns with greater than 0% missing values

In [20]:
# Get column names with missing values
null_columns = cols['column'].to_list()
null_columns

['veil-type',
 'spore-print-color',
 'stem-root',
 'veil-color',
 'stem-surface',
 'gill-spacing',
 'cap-surface',
 'gill-attachment',
 'ring-type',
 'gill-color',
 'habitat',
 'cap-shape',
 'stem-color',
 'has-ring',
 'cap-color',
 'does-bruise-or-bleed',
 'cap-diameter']

In [17]:
features = list(train.drop('class', axis=1).columns)
categorical_cols = list(train.select_dtypes(include='object').columns)[1:]
numeric_cols = list(set(features) - set(categorical_cols))
(features, categorical_cols, numeric_cols)

(['id',
  'cap-diameter',
  'cap-shape',
  'cap-surface',
  'cap-color',
  'does-bruise-or-bleed',
  'gill-attachment',
  'gill-spacing',
  'gill-color',
  'stem-height',
  'stem-width',
  'stem-root',
  'stem-surface',
  'stem-color',
  'veil-type',
  'veil-color',
  'has-ring',
  'ring-type',
  'spore-print-color',
  'habitat',
  'season'],
 ['cap-shape',
  'cap-surface',
  'cap-color',
  'does-bruise-or-bleed',
  'gill-attachment',
  'gill-spacing',
  'gill-color',
  'stem-root',
  'stem-surface',
  'stem-color',
  'veil-type',
  'veil-color',
  'has-ring',
  'ring-type',
  'spore-print-color',
  'habitat',
  'season'],
 ['cap-diameter', 'id', 'stem-width', 'stem-height'])

In [46]:
# correct values for columns from the original dataset
categories = {
    'cap-shape': ['b','c','x','f','k','s'],
    'cap-surface': ['f','g','y','s'],
    'cap-color': ['n','b','c','g','r','p','u','e','w','y'],
    'gill-attachment': ['a','d','f','n'],
    'gill-spacing': ['c','w','d'],
    'gill-color': ['k','n','b','h','g','r','o','p','u','e','w','y'],
    'veil-type': ['p','u'],
    'veil-color': ['n','o','w','y'],
    'ring-type': ['c','e','f','l','n','p','s','g'],
    'spore-print-color': ['k','n','b','h','r','o','u','w','y'],
    'habitat': ['g','l','m','p','u','w','d'],
    'stem-root': ['b','c','u','e','z','r',],
    'stem-surface': ['s','k','f','y'],
    'stem-color': ['k','n','b','h','r','o','u','w','y'],
    'has-ring': ['t','f'],
    'season': ['s','u','a','w'],
    'does-bruise-or-bleed': ['f','t'],

}

### Replace Wrong/Missing Values

In [54]:
def get_wrong_values(column):
  wrong_values = []
  for value in train[column].unique():
    if value not in categories[column]:
      wrong_values.append(value)
  return wrong_values

def replace_values(column, wrong_values):
  if train[column].dtype == 'object':
    for value in wrong_values:
      fill_value = random.choice(categories[column])
      train[column] = train[column].replace(value, fill_value)
  else:
    for value in wrong_values:
      fill_value = train[column].mean()
      train[column] = train[column].replace(value, fill_value)

def replace_missing_values(column):
  if train[column].dtype == 'object':
    fill_value = random.choice(categories[column])
    train[column].fillna(fill_value, inplace=True)
  if train[column].dtype == 'float64':
    train[column].fillna(train[column].mean(), inplace=True)

def convert(column):
  if train[column].dtype == 'object':
    train[column] = train[column].astype('category')
  else:
    train[column] = train[column].astype('numeric')

In [55]:
def transform():
  for column in categorical_cols:
    wrong_values = get_wrong_values(column)
    if len(wrong_values) > 0:
      replace_values(column, wrong_values)
    else:
      replace_missing_values(column)
    convert(column)

In [56]:
transform()

In [59]:
# test
def test_transform():
  for column in categorical_cols:
    wv = get_wrong_values(column)
    if len(wv) > 0:
      print(f'{column }failed')
    else:
      print(f'{column}: passed')

In [60]:
test_transform()

cap-shape: passed
cap-surface: passed
cap-color: passed
does-bruise-or-bleed: passed
gill-attachment: passed
gill-spacing: passed
gill-color: passed
stem-root: passed
stem-surface: passed
stem-color: passed
veil-type: passed
veil-color: passed
has-ring: passed
ring-type: passed
spore-print-color: passed
habitat: passed
season: passed


In [63]:
from scipy.stats import chi2_contingency

def find_p_value(df, col2='class', alpha=0.05):
  results = {}
  for col in df.columns:
    if col != col2 and df[col].dtype == 'category':
      contingency_table = pd.crosstab(df[col2], df[col])
      chi2, p, dof, expected = chi2_contingency(contingency_table)
      if p < alpha:
        results[col] = p
  return results

In [64]:
train_res = find_p_value(train)

In [67]:
train_df = train[train_res.keys()]

In [71]:
class_cat = train['class'].astype('category')

In [78]:
!pip install scikit-learn
from scikeras.wrappers import KerasClassifier
import keras

ImportError: cannot import name '_get_container_adapter' from 'sklearn.utils._set_output' (/usr/local/lib/python3.10/dist-packages/sklearn/utils/_set_output.py)

In [74]:
def get_model(meta):
    model = keras.models.Sequential()

    # Input layer and first Dense layer
    model.add(layers.Input(shape=(meta["X_shape_"][1],)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.3))

    # Second Dense layer
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.3))

    # Third Dense layer
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dropout(0.2))

    # Fourth Dense layer
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dropout(0.3))

    # Fifth Dense layer
    model.add(layers.Dense(4, activation='relu'))
    model.add(layers.Dropout(0.2))

    # Output layer
    model.add(layers.Dense(1, activation='sigmoid'))

    return model


display(keras.utils.plot_model(get_model({"X_shape_": processed_train.shape}),
                       show_shapes=True, show_layer_activations=True, dpi=80))


clf = KerasClassifier(
    get_model,
    loss="binary_crossentropy",
    optimizer=keras.optimizers.AdamW(learning_rate=0.01),
    validation_split=0.03,
    batch_size=8192,
    validation_batch_size=65536,
    epochs=50,
    verbose=1, # or 2
    callbacks=[keras.callbacks.ReduceLROnPlateau(patience=3),
               keras.callbacks.EarlyStopping(patience=5, min_delta=0.00003)]
)

NN_model = make_pipeline(ColumnTransformer([('float', make_pipeline(PowerTransformer(),
                                                                 SimpleImputer(add_indicator=True)),
                                          float_features),
                                         ('cat', OneHotEncoder(drop='first',
                                                               sparse_output=False,
                                                               dtype=np.float32,
                                                               handle_unknown='infrequent_if_exist',
                                                               min_frequency=50),
                                          cat_features),
                                        ]),
                      StandardScaler(),
                      BaggingClassifier(clf, n_estimators=7, bootstrap=False))

cross_validate(NN_model, 'Keras')

TypeError: data type 'numeric' not understood

In [72]:
from sklearn.model_selection import StratifiedKFold
import datetime
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import log_loss

In [73]:
oof, test_pred, fold_scores = {}, {}, {}

crossval_kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
init_features=sig_features
#init_features.remove('class')
def cross_validate(model, label,
                   features=init_features,
                   fit_params={},
                   train=processed_train,
                   plot_histogram=False,
                   use_eval_set=False):
    """Compute out-of-fold and test predictions for a given model.

    Out-of-fold and test predicted probabilities are stored in the global variables
    oof and test_pred, respectively, so that they can later be used for stacking
    the models in an ensemble.
    """
    start_time = datetime.datetime.now()
    global X_va, y_va
    scores = []
    oof_proba = np.full(len(train), np.nan, dtype=np.float32)
    for fold, (idx_tr, idx_va) in enumerate(crossval_kf.split(train, target)):
        X_tr = train.iloc[idx_tr][features]
        y_tr = targets_bool.iloc[idx_tr]
        X_va = train.iloc[idx_va][features]
        y_va = targets_bool.iloc[idx_va]


        if use_eval_set: fit_params['eval_set'] = [(X_va, y_va)]
        model.fit(X_tr, y_tr, **fit_params)

        y_pred_proba = model.predict_proba(X_va)[:,1]

        if plot_histogram:
            plt.figure(figsize=(8, 2))
            plt.hist(y_pred_proba, bins=500, density=True)
            plt.xlabel('predicted probability')
            plt.ylabel('density')
            plt.title('Histogram of predicted probabilities')
            plt.show()

        y_pred = y_pred_proba >= 0.5
        score = matthews_corrcoef(y_va, y_pred)
        print(f"# Fold {fold}: mcc={score:.5f}")
        scores.append(score)
        oof_proba[idx_va] = y_pred_proba

        break # one fold is enough

    elapsed_time = datetime.datetime.now() - start_time
    if not (np.isnan(oof_proba)).any():
        mcc = matthews_corrcoef(targets_bool, oof_proba >= 0.5)
        logloss = log_loss(targets_bool, oof_proba)
        print(f"{Fore.GREEN}# Overall: mcc={mcc:.5f}"
              f" logloss={logloss:.5f} {label}"
              f"   {int(np.round(elapsed_time.total_seconds() / 60))} min{Style.RESET_ALL}")
    else:
        print(f"{Fore.GREEN}# {label}"
              f"   {int(np.round(elapsed_time.total_seconds() / 60))} min{Style.RESET_ALL}")
    oof[label] = oof_proba
    fold_scores[label] = scores

    if COMPUTE_TEST_PRED:
        X_tr = train[features]
        y_tr = targets_bool

        if use_eval_set: del fit_params['eval_set']
        model.fit(X_tr, y_tr, **fit_params)
        test_pred[label] = model.predict_proba(test[features])[:,1]

NameError: name 'sig_features' is not defined